In [1]:
import torch
import torch.nn as nn
import torch_sparse
import torchsparse.nn.functional as F
from torchsparse import SparseTensor
from torchsparse.nn import SparseConv3d

# Minimal example data
coordinates = torch.tensor([
    [0, 0, 0, 0],
    [0, 1, 1, 1],
    [0, 2, 2, 2],
    [0, 3, 3, 3]
], dtype=torch.int32)
features = torch.tensor([
    [1.0],
    [2.0],
    [3.0],
    [4.0]
], dtype=torch.float32)

# Create sparse tensor
sparse_tensor = SparseTensor(features, coordinates)

# Verify the sparse tensor creation
print("Coordinates:", coordinates)
print("Features:", features)
print("SparseTensor Feature Shape:", sparse_tensor.F.shape)
print("SparseTensor Coordinate Shape:", sparse_tensor.C.shape)

# Define a simple sparse conv network
class SimpleSparseConvNet(nn.Module):
    def __init__(self):
        super(SimpleSparseConvNet, self).__init__()
        self.conv1 = SparseConv3d(1, 4, kernel_size=2, stride=1)

    def forward(self, x):
        print(f"Input: features shape {x.F.shape}, coordinates shape {x.C.shape}")
        x = self.conv1(x)
        print(f"Output: features shape {x.F.shape}, coordinates shape {x.C.shape}")
        return x

# Initialize the model
model = SimpleSparseConvNet()

# Forward pass on CPU
try:
    output = model(sparse_tensor.cpu())
    print("Forward pass successful")
    print("Output features:", output.F)
    print("Output coordinates:", output.C)
except RuntimeError as e:
    print(f"RuntimeError during forward pass: {e}")
    import traceback
    traceback.print_exc()
except NotImplementedError as e:
    print(f"NotImplementedError during forward pass: {e}")
    import traceback
    traceback.print_exc()
except Exception as e:
    print(f"Unexpected error: {e}")
    import traceback
    traceback.print_exc()


: 

In [13]:
class SimpleDenseConvNet(nn.Module):
    def __init__(self):
        super(SimpleDenseConvNet, self).__init__()
        self.conv1 = nn.Conv3d(1, 4, kernel_size=2, stride=1)

    def forward(self, x):
        print(f"Input: shape {x.shape}")
        x = self.conv1(x)
        print(f"Output: shape {x.shape}")
        return x

# Convert sparse tensor to dense format for testing
dense_tensor = torch.zeros((1, 1, 4, 4, 4))  # Adjust shape as necessary
for idx, coord in enumerate(coordinates):
    dense_tensor[0, 0, coord[1], coord[2], coord[3]] = features[idx]

# Initialize the model
dense_model = SimpleDenseConvNet()

# Forward pass
try:
    dense_output = dense_model(dense_tensor)
    print("Dense forward pass successful")
    print("Output shape:", dense_output.shape)
except RuntimeError as e:
    print(f"RuntimeError during dense forward pass: {e}")
    traceback.print_exc()
except Exception as e:
    print(f"Unexpected error: {e}")
    traceback.print_exc()


Input: shape torch.Size([1, 1, 4, 4, 4])
Output: shape torch.Size([1, 4, 3, 3, 3])
Dense forward pass successful
Output shape: torch.Size([1, 4, 3, 3, 3])


In [3]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


PyTorch version: 2.3.1+cu121
CUDA available: True
CUDA version: 12.1
